In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
#Testing api key
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key=os.getenv("GOOGLE-API-KEY"),
    temperature=0.3
)

response = llm.invoke("Say hello in Python")
print(response)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    google_api_key = os.getenv("GOOGLE-API-KEY")
)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain,SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [ ]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text your job is to create {number} multiple choice questions in {tone} tone.
Make sure questions are not repeated and make sure that they follow the text above.
Make sure to format the response on basis of RESPONSE_JSON below.
##RESPONSE_JSON
{response_json} 
"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [ ]:
RESPONSE_JSON = {
    "1":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
    "2":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
    "3":{
        "mcq":"multiple choice qustion",
        "options":{
            "a":"option 1",
            "b":"option 2",
            "c":"option 3",
            "d":"option 4"
        },
        "answer":"correct answer",
    },
}

In [ ]:
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import RunnableSequence
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

quiz_chain = RunnableParallel({
    "quiz": quiz_generation_prompt | llm,
    "subject": RunnablePassthrough()
})

In [ ]:
TEMPLATE2 = """
Youo are an expert english grammarian and writer. Given a multiple choice Quiz for {subject} students.
You need to evaluate the complexity of the quiz questions, and also check wheather is quiz is adapt to the quantitstive and cognitive ability of students.
If not yiu need to change the questions and change the tone so that it perfectly fits the students abilities.
QUIZ MCQ:{quiz}

Check as an expert english Writer for the above quiz.
"""

In [ ]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=['subject','quiz'],
    template=TEMPLATE2
)

In [ ]:
review_chain = RunnableSequence(
    quiz_evaluation_prompt,
    llm,
    RunnableLambda(lambda x: {"review": x})
)

In [ ]:
from langchain_core.runnables import RunnableSequence

generate_evaluate_chain = quiz_chain | review_chain

In [ ]:
file_path = "P:\Code\Gen_AI_course_ineuron\MCQ_GEN_AI\data.txt"
with open(file_path,"r") as file:
    TEXT = file.read()

In [ ]:
print(TEXT)

In [ ]:
json.dumps(RESPONSE_JSON)

In [ ]:
NUMBER = 5
SUBJECT = "python"
TONE = "simple"

In [ ]:
from contextlib import contextmanager

@contextmanager
def get_gemini_callback():
    usage_log = {
        "prompt_tokens": 0,
        "completion_tokens": 0,
        "total_tokens": 0,
        "total_cost": 0.0
    }
    yield usage_log
    # You can log or return this after yield

In [ ]:
quiz_output = quiz_chain.invoke(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)

In [ ]:
with get_gemini_callback() as gc:
    response = generate_evaluate_chain.invoke(
        {
            "text":TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
print(response)

In [ ]:
response

In [ ]:
quiz_output

In [ ]:
res = quiz_output.get("quiz")
res

In [ ]:
quiz_string = res.content
cleaned_string = quiz_string.strip("```json\n").strip("`")
final_mcqs = json.loads(cleaned_string)
final_mcqs
print(json.dumps(final_mcqs, indent=4))

In [ ]:
quiz_table_data = []
for key,value in final_mcqs.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    answer = value["answer"]
    quiz_table_data.append({"MCQ":mcq,"Choices":options,"Answer":answer})


In [ ]:
quiz_table_data

In [ ]:
quiz = pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
import pandas as pd
import os
import json
import traceback
from dotenv import load_dotenv
load_dotenv()

from src.mcqgenerator.utils import read_file,get_table_data
from src.mcqgenerator.looger import logging

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage,HumanMessage
from langchain.chains import LLMChain

llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    google_api_key = os.getenv("GOOGLE-API-KEY"),
    temperature = 0.7,
)

quiz_generation_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content = "You are an expert MCQ quiz maker. You have to create the quiz based on user input and given instructions."
        ),
        HumanMessage(
            content = """
                Please create {number} muliple choice questions in a {tone} tone based on the following text.
                **Text:**
                {text}
                
                **Instructions:**
                1. Ensure that all questions are from given text only.
                2. Ensure that the questions are not repeated.
                3. Format your entire response as a JSON object that strictly follows below given scheme.
                {response_json}
            """
        )
    ]    
)

quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, ouput_key="quiz",verbose=True)

res = quiz_chain.invoke()

In [ ]:
from langchain_google_genai import __all__
print(__all__)

In [ ]:
from langchain.callbacks import __all__
print(__all__)

In [ ]:
import os
import json
import traceback
import streamlit as st
import pandas as pd
from src.mcqgenerator.utils import read_file,get_table_data
from src.mcqgenerator.MCQGenerator import genereate_evaluate_chain
from src.mcqgenerator.looger import logging
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
load_dotenv()

with open('Response.json','r') as file:
    RESPONSE_JSON = json.load(file)

st.title("MCQ generator using LangChain")

with st.form("user_inputs"):
    uploaded_file = st.file_uploader("Upload a PDF or TEXT file")
    mcq_count = st.number_input("No. of MCQ",min_value=3,max_value=50)
    subject = st.text_input("Enter subject",max_chars=20)
    tone = st.text_input("Complexity level of Questions",max_chars=20,placeholder="Simple")
    button = st.form_submit_button("Create MCQ")


if button and uploaded_file is not None and mcq_count and subject and tone:
    with st.spinner("Loading...."):
        try:
            text = read_file(uploaded_file)
            with get_openai_callback() as cb:
                response = genereate_evaluate_chain.invoke(
                    {
                        "text": text,
                        "number":mcq_count,
                        "subject": subject,
                        "tone":tone,
                        "response_json": RESPONSE_JSON
                    }
                )
        except Exception as e:
            traceback.print_exception(type(e),e,e.__traceback__)
            st.error("Error")

        else:
            print(f"Total Tokens Used: {cb.total_tokens}")
            print(f"Prompt Tokens: {cb.prompt_tokens}")
            print(f"Completion Tokens: {cb.completion_tokens}")
            print(f"Total Cost (USD): ${cb.total_cost:.4f}")
            if isinstance(response,dict):
                quiz = response.get("quiz",None)    
                if quiz is not None:
                    table_data = get_table_data(quiz)
                    if table_data is not None:
                        df = pd.DataFrame(table_data)
                        df.index += 1
                        st.table(df)
                        st.text_area(label="Review",value=response["review"])
                    else:
                        st.error("Error in the table data")
            else:
                st.write(response)
